In [3]:
import numpy as np
import torch
import torch.nn as nn


In [49]:

xyz = torch.rand(3,100,3)
features = torch.rand(3,100,6)
npoint=50
nsample = 25
mlp = [32,32,64]
new_xyz = torch.rand(3,npoint,3)
grouped_xyz = torch.rand(3,npoint,nsample,3)
grouped_features = torch.rand(3,npoint,nsample,6)
print(grouped_features.shape)
density_scale = torch.rand(3,npoint, nsample,1)


torch.Size([3, 50, 25, 6])


In [50]:
grouped_features2 = grouped_features.permute(0,3,2,1)
print(grouped_features2.shape)
for ind,i in enumerate(mlp):
    if ind != len(mlp)-1:
        print(ind)
        grouped_features2 = nn.Conv2d(grouped_features2.shape[1], i, 1, padding='valid')(grouped_features2)
        grouped_features2 = nn.BatchNorm2d(i)(grouped_features2)
        grouped_features2 = nn.ReLU()(grouped_features2)

    # print(grouped_features2.shape)
grouped_features2 = grouped_features2.permute(0,3,2,1)
print(grouped_features2.shape)

torch.Size([3, 6, 25, 50])
0
1
torch.Size([3, 50, 25, 32])


In [51]:
print(grouped_xyz.shape)
grouped_xyz1 = grouped_xyz.permute(0,3,2,1)
for i in [32]:
    weights = nn.Conv2d(grouped_xyz1.shape[1], i, 1, padding='valid')(grouped_xyz1)
    weights = nn.BatchNorm2d(i)(weights)
    weights = nn.ReLU()(weights)
weights = weights.permute(0,3,2,1)
print(weights.shape)

torch.Size([3, 50, 25, 3])
torch.Size([3, 50, 25, 32])


In [52]:
density_scale = torch.rand(3,npoint, nsample,1)
print(density_scale.shape)
density_scale = density_scale.permute(0,3,2,1)
print(density_scale.shape)
for i in [16,1]:
    x= nn.Conv2d(density_scale.shape[1], i, 1, padding='valid')(density_scale)
    x = nn.BatchNorm2d(i)(x)
    x = nn.ReLU()(x)
    print(x.shape)
density_scale = x
density_scale = density_scale.permute(0,3,2,1)
print(density_scale.shape)

torch.Size([3, 50, 25, 1])
torch.Size([3, 1, 25, 50])
torch.Size([3, 16, 25, 50])
torch.Size([3, 1, 25, 50])
torch.Size([3, 50, 25, 1])


In [53]:
new_points = grouped_features2*density_scale
print(new_points.shape)

torch.Size([3, 50, 25, 32])


In [54]:
new_points = torch.matmul(new_points.permute(0,1,3,2), weights)
new_points.shape

torch.Size([3, 50, 32, 32])

In [55]:
new_points.shape

torch.Size([3, 50, 32, 32])

In [56]:
new_points2 = new_points.permute(0,1,3,2)
print(new_points2.shape)

torch.Size([3, 50, 32, 32])


In [58]:
new_points2 = new_points.permute(0,3,2,1)
print(new_points2.shape)
new_points2 = nn.Conv2d(new_points2.shape[1], mlp[-1], (new_points2.shape[2],1), padding='valid')(new_points2)
print(new_points2.shape)
# new_points = new_points.permute(0,1,3,2)
# new_points = new_points.squeeze(2)

torch.Size([3, 32, 32, 50])
torch.Size([3, 64, 1, 50])


In [59]:
new_points2 = new_points2.permute(0,3,2,1)
print(new_points2.shape)
new_points2 = new_points2.squeeze(2)
print(new_points2.shape)


torch.Size([3, 50, 1, 64])
torch.Size([3, 50, 64])
